# Process Satellite Data with Dask

Process large raster file.

In [ ]:
import os
# import rasterio
import matplotlib.pyplot as plt
# import xarray as xr
import rioxarray as rxr
import numpy as np

In [ ]:
%matplotlib inline

## Set up Dask client

In [ ]:
from dask.distributed import Client, progress
# # client = Client(n_workers=2, threads_per_worker=2, memory_limit='10GB')
client = Client(n_workers=4, threads_per_worker=2, memory_limit='10GB')
# client = Client('tcp://193.2.92.144:8786')
client

## Load data

In [ ]:
%%time
s2_fn_ir = 'C:\Data\GeoData\Sentinel-2\S2B_MSIL2A_20200822T094039_N0214_R036_T34UDC_20200822T115325.SAFE\GRANULE\L2A_T34UDC_A018080_20200822T094034\IMG_DATA\R10m\T34UDC_20200822T094039_B08_10m.jp2'
s2_fn_red = 'C:\Data\GeoData\Sentinel-2\S2B_MSIL2A_20200822T094039_N0214_R036_T34UDC_20200822T115325.SAFE\GRANULE\L2A_T34UDC_A018080_20200822T094034\IMG_DATA\R10m\T34UDC_20200822T094039_B04_10m.jp2'

In [ ]:
%%time
nir = rxr.open_rasterio(s2_fn_ir, chunks={'band': 1, 'x': 1024, 'y': 1024}, masked=True).astype(np.float64)
red = rxr.open_rasterio(s2_fn_red, chunks={'band': 1, 'x': 1024, 'y': 1024}, masked=True).astype(np.float64)

In [ ]:
nir

In [ ]:
nir.shape

In [ ]:
nir.dims

In [ ]:
sub = nir[:,[1000, 2000], [1000, 2000]]

In [ ]:
sub.shape

In [ ]:
%%time
plt.figure()
nir[0,:,:].plot.imshow(cmap='BrBG')
plt.axis('equal')
plt.show()

## Compute NDVI

In [ ]:
%%time
ndvi = (nir - red) / (nir + red)

In [ ]:
%%time
ndvi2d = ndvi.squeeze()

In [ ]:
ndvi2d

In [ ]:
%%time
plt.figure()
ndvi2d.compute().plot.imshow(cmap='BrBG', vmin=-0.5, vmax=1)
plt.axis('equal')
plt.show()

## Create a subset

In [ ]:
ndvi2d.sel

In [ ]:
sub = ndvi2d[0:10, 0:20]

In [ ]:
sub = ndvi.sel(lon=(ds.lon < -80) | (ds.lon > 40))

In [ ]:
plt.figure()
sub.compute().plot.imshow(cmap='BrBG', vmin=-0.5, vmax=1)
plt.axis('equal')
plt.show()